In [1]:
import pandas as pd
import numpy as np

train_ratings = pd.read_csv("train_ratings.csv")
val_ratings = pd.read_csv("val_ratings.csv")

In [2]:
num_users = int(train_ratings['userId'].max() + 1) #138494
num_movies = int(train_ratings['movieId'].max() + 1) #120467
EMBEDDINGS = 50 #hyperparameter

train_X = [train_ratings['userId'], train_ratings['movieId']]
train_y = train_ratings['rating']
val_X = [val_ratings['userId'], val_ratings['movieId']]
val_y = val_ratings['rating']

In [3]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import *
#http://www.fenris.org/2016/03/07/index-html
User = Sequential()
Movie = Sequential()
User.add(Embedding(num_users, EMBEDDINGS, input_length=1))
Movie.add(Embedding(num_movies, EMBEDDINGS, input_length=1))

User.add(Flatten())
Movie.add(Flatten())

#Added = Add()([User.output, Movie.output])
Output = dot([User.output, Movie.output], axes=1)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

model = Model(inputs=[User.input, Movie.input], outputs=[Output])
model.compile(loss='mse', optimizer='adamax') #https://www.quora.com/When-would-you-use-Adamax-over-Adam

In [5]:
callbacks = [EarlyStopping('val_loss', patience=3), ModelCheckpoint("best_model.hdf5", save_best_only=True)]
history = model.fit(train_X, train_y, epochs=30, validation_data=(val_X, val_y), verbose=2, callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Train on 11946576 samples, validate on 3999236 samples
Epoch 1/30
 - 1960s - loss: 5.6640 - val_loss: 3.9975
Epoch 2/30
 - 2339s - loss: 1.4367 - val_loss: 2.5804
Epoch 3/30
 - 1950s - loss: 1.0485 - val_loss: 2.0664
Epoch 4/30
 - 1947s - loss: 0.9266 - val_loss: 1.7836
Epoch 5/30
 - 1947s - loss: 0.8714 - val_loss: 1.5991
Epoch 6/30
 - 1947s - loss: 0.8402 - val_loss: 1.4684
Epoch 7/30
 - 1945s - loss: 0.8191 - val_loss: 1.3720
Epoch 8/30
 - 1943s - loss: 0.8029 - val_loss: 1.2955
Epoch 9/30
 - 1943s - loss: 0.7884 - val_loss: 1.2343
Epoch 10/30
 - 1944s - loss: 0.7744 - val_loss: 1.1830
Epoch 11/30
 - 1943s - loss: 0.7592 - val_loss: 1.1385
Epoch 12/30
 - 1942s - loss: 0.7435 - val_loss: 1.1007
Epoch 13/30
 - 1943s - loss: 0.7286 - val_loss: 1.0676
Epoch 14/30
 - 1943s - loss: 0.7144 - val_loss: 1.0379
Epoch 15/30
 - 1943s - loss: 0.7009 - val_loss: 1.0131
Epoch 16/30
 - 1943s - loss: 0.6883 - val_loss: 0.9903
Epoch 17/30
 - 1942s - los

In [6]:
#latest model
test_ratings = pd.read_csv("test_ratings.csv")
predictions = model.predict([test_ratings['userId'], test_ratings['movieId']], verbose=1)

4054451/4054451 [==============================] - 98s 24us/step


In [7]:
pred_df = pd.DataFrame(predictions, columns=['rating'])
result = pd.concat([test_ratings["Id"], pred_df], axis=1, sort=False)
result.to_csv('keras_2_latest_result.csv', columns=['Id', 'rating'], index = None)

In [8]:
#rounded to nearest 0.5
pred_df = pd.DataFrame(np.around(predictions * 2) / 2, columns=['rating'])
result = pd.concat([test_ratings["Id"], pred_df], axis=1, sort=False)
result.to_csv('keras_2_latest_result_rounded.csv', columns=['Id', 'rating'], index = None)